Data Preprocessing 

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder, HashingEncoder

df = pd.read_csv('Clean_Dataset.csv')
one_hot = OneHotEncoder()
target = TargetEncoder()
hash = HashingEncoder(n_components=16)


airline_encoded = one_hot.fit_transform(df[["airline"]]).toarray()
airline_labels =  [f"airline_{cat}" for cat in one_hot.categories_[0]]
airline_df = pd.DataFrame(airline_encoded, columns=airline_labels)

source_encoded = one_hot.fit_transform(df[["source_city"]]).toarray()
source_labels = [f"source_{cat}" for cat in one_hot.categories_[0]]
source_df = pd.DataFrame(source_encoded, columns=source_labels)

destination_encoded = one_hot.fit_transform(df[["destination_city"]]).toarray()
destination_labels = [f"destination_{cat}" for cat in one_hot.categories_[0]]
destination_df = pd.DataFrame(destination_encoded, columns=destination_labels)

flight_encoded = hash.fit_transform(df['flight'])

df["departure_time"]= target.fit_transform(df['departure_time'], df['price'])
df["arrival_time"]= target.fit_transform(df['arrival_time'], df['price'])

df['stops'] = df['stops'].map({'zero':0, 'one':1, 'two_or_more':2})
df['class'] = df['class'].map({'Economy':0, 'Business':1})

new_df = pd.concat([df,flight_encoded, airline_df, source_df, destination_df], axis=1)
new_df = new_df.drop(["Unnamed: 0", "flight", "airline", "source_city", "destination_city"], axis=1)
new_df = new_df.astype('float32')




Splitting Data for Future Prediction

In [4]:
future_df = new_df.tail(15)
training_df = new_df.iloc[:-15]

Random Selection

In [6]:
import random

# Group columns into categories
airline_columns = airline_df.columns.tolist()
source_columns = source_df.columns.tolist()
destination_columns = destination_df.columns.tolist()
flight_columns = flight_encoded.columns.tolist()

# Add groups to a list
groups = {
    "airline": airline_columns,
    "source_city": source_columns,
    "destination_city": destination_columns,
    "flight": flight_columns,
}

# Flatten the remaining individual columns in `new_df`
individual_columns = [
    col
    for col in training_df.columns
    if col not in airline_columns + source_columns + destination_columns + flight_columns
]
individual_columns.remove("price")

# Combine groups with individual columns for selection
all_features = list(groups.keys()) + individual_columns

# Randomly select 5 features
select_features = random.sample(all_features, 4)

# Expand selected features into their full groups if necessary
final_features = []
for feature in select_features:
    if feature in groups:  # If the feature is a group, include all its columns
        final_features.extend(groups[feature])
    else:  # Otherwise, it's an individual column
        final_features.append(feature)

# Create a new DataFrame with the selected features
selected_features = training_df[final_features]

# Print selected features
print("Selected features:", [
        feature if feature in groups else feature for feature in select_features])

Selected features: ['departure_time', 'flight', 'days_left', 'airline']


Dataset Splitting

In [7]:
from sklearn.model_selection import train_test_split

x = selected_features  
y = training_df['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Model 1: XGBoost

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Initialize the XGBoost regressor
xgb_model = XGBRegressor(
    n_estimators=100,       # Number of trees
    max_depth=6,            # Maximum depth of trees
    learning_rate=0.1,      # Learning rate
    subsample=0.8,          # Subsampling ratio
    colsample_bytree=0.8,   # Feature subsampling ratio per tree
    random_state=42
)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")

# Feature importance (optional)
import matplotlib.pyplot as plt
from xgboost import plot_importance

plt.figure(figsize=(10, 8))
plot_importance(xgb_model, importance_type='weight', show_values=False)
plt.title('Feature Importance')
plt.show()


ModuleNotFoundError: No module named 'xgboost'

Model 2: Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=50, max_depth=10,
    max_features='sqrt',
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42)
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=10,
                       n_estimators=50, random_state=42)

Model 3: ANN 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

ann_model = Sequential([
    Dense(64, activation="relu", input_dim=X_train.shape[1]),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")  
])
ann_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
ann_accuracy = ann_model.evaluate(X_test, y_test, verbose=0)[1]
print(f"ANN Accuracy: {ann_accuracy}")
